In [1]:
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
df_bank = pd.read_csv('data_2/bank_accounts.csv')
df_credit = pd.read_csv('data_2/credit_cards.csv')
df_order = pd.read_csv('data_2/orders.csv')
df_devices = pd.read_csv('data_2/devices.csv')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
%%time
g = nx.Graph()

print("adding edges for credit cards..")
for idx, ut in df_credit.iterrows():
    if idx % 100000 == 0:
        print(idx)
    u, t = ut
    t = 'credit_card:' + t
    g.add_edge(u, t,weight = 1)

adding edges for credit cards..
0
CPU times: user 6.44 s, sys: 65 ms, total: 6.51 s
Wall time: 6.55 s


In [4]:
%%time
print("adding edges for bank accounts..")
for idx, ut in df_bank.iterrows():
    if idx % 100000 == 0:
        print(idx)
    u, t = ut
    t = 'bank_account:' + str(t)
    g.add_edge(u, t, weight=2)

adding edges for bank accounts..
0
100000
200000
300000
CPU times: user 1min 3s, sys: 849 ms, total: 1min 4s
Wall time: 1min 7s


In [5]:
%%time
print("adding edges for devices..")
for idx, ud in df_devices.iterrows():
    if idx % 100000 == 0:
        print(idx)
    u, t = ud
    t = 'device:' + t
    g.add_edge(u, t,weight=3)

adding edges for devices..
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
CPU times: user 4min 20s, sys: 4.9 s, total: 4min 25s
Wall time: 4min 35s


# Networkx with DFS

In [159]:
def dfs(buyer, seller):
    visited=[]
    queue = [[buyer]]
    link=[]
    shortest_link = np.inf
    w = []
    while len(queue)!=0:
        path = queue.pop(0)
        #print(path)
        v = path[-1]
        if (v == seller):
            if (len(path)<= shortest_link):
                shortest_link = len(path)
                link.append(path)
                weight = 0
                for idx in range(len(path)-1):
                    val = path[idx]
                    val2 = path[idx+1]
                    weight += g[val][val2]['weight']
                w.append(weight)
        elif v not in visited:
            all_n = nx.neighbors(g,v)
            for n in all_n:
                new_path = list(path)
                new_path.append(n)
                queue.append(new_path)
            visited.append(v)
    
    if (len(link)>0):
        link = min(link)
        #print(link)
    return link
    

In [161]:
%%time
with open('data_2/out_nx_dfs.csv', 'w') as f:
    f.write('idx,orderid,is_fraud\n')
    for idx, order in df_order.iterrows():
            #print(idx)
            if idx % 50000 == 0:
                print(idx)
            oid, buyer, seller = order
            if buyer == seller:
                print(str(idx)+','+str(oid) + ',' + str(buyer) + '\n')
                continue

            try:
                path = dfs(buyer, seller)
                if len(path) < 1:
                    f.write(str(idx)+','+str(oid) + ',not fraud\n')
                #    print ("more than one path" + str(idx))
                
                #print(type(path))
                #print(path[1:-1])
                else: 
                    toprint = str(idx)+','+str(oid) + ','+ str(buyer)+'-'
                    #print(str(oid) + ','+ str(buyer)+'-')
                    for p in path[1:-1]:
                        if isinstance(p,str):
                            toprint = toprint + '"' + str(p) + '"->'
                        else:
                            toprint = toprint + str(p) + '-'
                    toprint = toprint + str(seller) + '\n'
                    f.write(toprint)
                
            except nx.NetworkXNoPath:
                f.write(str(idx)+','+str(oid) + ',not fraud\n')
                #continue
            except nx.NodeNotFound:
                f.write(str(idx)+','+str(oid) + ',not fraud\n')
                #continue
            except nx.NetworkXError:
                f.write(str(idx)+','+str(oid) + ',not fraud\n')
                #continue
        

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
CPU times: user 55min 9s, sys: 6.05 s, total: 55min 15s
Wall time: 1h 2min 10s
